# Installing dependencies


In [1]:
!pip install geopandas
!pip install plotly_express
!pip install tqdm
!pip install geopy

     |████████████████████████████████| 931kB 5.0MB/s 
     |████████████████████████████████| 10.9MB 32.5MB/s 
     |████████████████████████████████| 14.7MB 326kB/s 


# Importing libraries and packages

In [2]:

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
from google.colab import drive 
drive.mount('/content/gdrive')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Function that converts latitude and longitude into respective zip code using GeoPy OpenStreetMap's Nominatim API service


In [0]:
def reversegeo(lat, lon):
  locator = Nominatim(user_agent="myGeocoder")
  coordinates = str(lat)+","+str(lon)
  location = locator.reverse(coordinates)
  if "postcode" in location.raw['address']:
    return location.raw['address']['postcode']
  else: return "NA"

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls "/content/drive/My Drive/Mobility data"

202003-citibike-tripdata.csv  march_citiends.csv  march_citistarts.csv


In [0]:
cititrips=pd.read_csv("/content/drive/My Drive/Mobility data/202003-citibike-tripdata.csv")

# Citi-bike end stations
## Extracting the end station of a bike trip along with its latitude and longitude

In [0]:
citiends=cititrips[['end station id','end station latitude', 'end station longitude']]
temp1=citiends['end station id'].value_counts().reset_index()
temp1.columns=['end station id','tripcount']
temp1=pd.merge(temp1, citiends, how='inner', on='end station id')
temp1=temp1.drop_duplicates()

## Applying the reversegeocoding function to obtain zipcode for end station

In [0]:
temp1['zipcode']=temp1.apply(lambda x: reversegeo(x['end station latitude'], x['end station longitude']), axis=1)

In [10]:
temp1.head()

,end station id,tripcount,end station latitude,end station longitude,zipcode
0,519,6538,40.751873,-73.977706,10037
2782,519,6538,40.752000,-73.977000,10017
6538,497,6109,40.737050,-73.990093,10010
12647,435,6008,40.741740,-73.994156,10011
18655,426,5758,40.717548,-74.013221,10282


## Truncating zipcodes longer than 5 digits to 5 digits

In [0]:
bike_end=temp1
bike_end['zipcode']=bike_end['zipcode'].apply(lambda x: x[:5])


In [17]:
bike_end.head()

,end station id,tripcount,end station latitude,end station longitude,zipcode
0,519,6538,40.751873,-73.977706,10037
2782,519,6538,40.752000,-73.977000,10017
6538,497,6109,40.737050,-73.990093,10010
12647,435,6008,40.741740,-73.994156,10011
18655,426,5758,40.717548,-74.013221,10282


## Removing zip codes which are NA and dropping the latitude,longitude, and station_id columns

In [0]:
bike_end=bike_end[bike_end['zipcode']!='NA']
bike_end=bike_end.drop(['end station latitude','end station longitude'],axis=1)

In [0]:
march_Citi_ends=bike_end.drop(['end station id'], axis=1)

## Summing up the trips ending in a particular zipcode

In [0]:
march_Citi_ends=march_Citi_ends.groupby(['zipcode'])['tripcount'].sum().reset_index()

In [29]:
march_Citi_ends.head()

,zipcode,tripcount
0,07302,8
1,07304,1
2,07307,2
3,07310,2
4,10000,2494


## Generating CSV

In [0]:
march_Citi_ends.to_csv('march_citi_end.csv', index=False)

# Citi-bike start stations
Same process is repeating for stations citibike starts from

In [0]:
citistarts=cititrips[['start station id','start station latitude', 'start station longitude']]
temp1=citistarts['start station id'].value_counts().reset_index()
temp1.columns=['start station id','tripcount']
temp1=pd.merge(temp1, citistarts, how='inner', on='start station id')
temp1=temp1.drop_duplicates()

In [0]:
temp1['zipcode']=temp1.apply(lambda x: reversegeo(x['start station latitude'], x['start station longitude']), axis=1)

In [24]:
temp1.head()

,start station id,tripcount,start station latitude,start station longitude,zipcode
0,519,6618,40.751873,-73.977706,10037
2900,519,6618,40.752000,-73.977000,10017
6618,435,5949,40.741740,-73.994156,10011
12567,426,5733,40.717548,-74.013221,10282
14672,426,5733,40.719000,-74.013000,10282


In [26]:
bike_start=temp1
bike_start['zipcode']=bike_start['zipcode'].apply(lambda x: x[:5])
bike_start.head()

,start station id,tripcount,start station latitude,start station longitude,zipcode
0,519,6618,40.751873,-73.977706,10037
2900,519,6618,40.752000,-73.977000,10017
6618,435,5949,40.741740,-73.994156,10011
12567,426,5733,40.717548,-74.013221,10282
14672,426,5733,40.719000,-74.013000,10282


In [0]:
bike_start=bike_start[bike_start['zipcode']!='NA']
bike_start=bike_start.drop(['start station latitude','start station longitude'],axis=1)

In [30]:
march_Citi_starts=bike_start.drop(['start station id'], axis=1)
march_Citi_starts=march_Citi_starts.groupby(['zipcode'])['tripcount'].sum().reset_index()
march_Citi_starts.to_csv('march_citi_starts.csv', index=False)
march_Citi_starts.head()

,zipcode,tripcount
0,10000,2547
1,10001,49109
2,10002,61246
3,10003,56741
4,10004,6333
